## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-25-05-49-19 +0000,bbc,Drone activity confirmed at multiple Denmark a...,https://www.bbc.com/news/articles/c7401vk4lgzo...
1,2025-09-25-05-43-55 +0000,bbc,BBC and news agencies launch film calling on I...,https://www.bbc.com/news/articles/c8d7yrp80m8o...
2,2025-09-25-05-25-51 +0000,nyt,3 Strong Earthquakes Strike Venezuela,https://www.nytimes.com/2025/09/25/world/ameri...
3,2025-09-25-05-05-24 +0000,nypost,Owners of Radio Shack and Pier 1 Imports accus...,https://nypost.com/2025/09/25/us-news/owners-o...
4,2025-09-25-04-45-50 +0000,nypost,Megyn Kelly schools student over his ‘blatant’...,https://nypost.com/2025/09/25/us-news/megyn-ke...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2391/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
146,trump,58
248,ice,16
110,new,15
99,china,13
301,kimmel,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
302,2025-09-24-09-30-00 +0000,wsj,Stephen Miran set out his case for lower inter...,https://www.wsj.com/economy/central-banking/st...,107
239,2025-09-24-14-05-20 +0000,nypost,Trump threatens to sue ABC over Jimmy Kimmel’s...,https://nypost.com/2025/09/24/us-news/trump-th...,106
121,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com,104
90,2025-09-24-22-55-47 +0000,nypost,Trump’s pro-Ukraine shift a ‘strategic move’ b...,https://nypost.com/2025/09/24/world-news/trump...,100
108,2025-09-24-22-15-02 +0000,latimes,Trump's attacks on Kimmel and ABC put him at o...,https://www.latimes.com/politics/story/2025-09...,93


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
302,107,2025-09-24-09-30-00 +0000,wsj,Stephen Miran set out his case for lower inter...,https://www.wsj.com/economy/central-banking/st...
105,70,2025-09-24-22-20-12 +0000,nypost,"Vance tears into Democrats, calls out Gavin Ne...",https://nypost.com/2025/09/24/us-news/vance-te...
109,57,2025-09-24-22-12-30 +0000,latimes,Jimmy Kimmel's return scores 6.3 million viewe...,https://www.latimes.com/entertainment-arts/bus...
322,55,2025-09-24-07-04-00 +0000,wsj,China said it will no longer seek new special ...,https://www.wsj.com/economy/trade/china-to-sto...
228,43,2025-09-24-15-06-31 +0000,bbc,Israeli forces near Gaza City centre as Hamas ...,https://www.bbc.com/news/articles/c4gk9dreldxo...
25,38,2025-09-25-02-51-33 +0000,nypost,"Mom who lost daughter, 15, to suicide after on...",https://nypost.com/2025/09/24/world-news/mom-w...
38,36,2025-09-25-01-34-30 +0000,nypost,Trans boyfriend of Charlie Kirk suspect Tyler ...,https://nypost.com/2025/09/24/us-news/trans-bo...
7,34,2025-09-25-04-11-00 +0000,wsj,The Office of Management and Budget told feder...,https://www.wsj.com/politics/policy/white-hous...
50,33,2025-09-25-01-00-00 +0000,wsj,RFK Jr.’s team wanted to tout an autism therap...,https://www.wsj.com/health/healthcare/tylenol-...
259,33,2025-09-24-12-19-49 +0000,bbc,Typhoon Ragasa makes landfall in China after 1...,https://www.bbc.com/news/articles/clyd580lx3mo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
